# Quickstart for beginners

Import Tensorflow.

In [1]:
import tensorflow as tf

Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers.

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


Build the ```tf.keras.Sequential``` model by stacking layers. Choose an optimizer and loss function for training.

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of 'logits' or 'log-odds' scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.8947428 , -0.04675238,  0.23789485, -0.22301272,  0.27142102,
        -0.10838422,  0.22652109, -0.62980294,  0.2963757 ,  0.36151463]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to 'probabilities' for each class.

Note: it is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.04003756, 0.09348571, 0.1242699 , 0.07837839, 0.12850682,
        0.08789797, 0.12286448, 0.05218307, 0.13175401, 0.14062202]],
      dtype=float32)

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: it is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`

In [9]:
loss_fn(y_train[0], predictions).numpy()

2.4315786

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss.

In [12]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3016 - accuracy: 0.9125
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1458 - accuracy: 0.9565
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1082 - accuracy: 0.9666
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0875 - accuracy: 0.9732
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0767 - accuracy: 0.9761


The `Model.evaluate` method checks the models performance, usually on a 'validation-set' or 'test-set'.

In [14]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0752 - accuracy: 0.9769


[0.07522686570882797, 0.9768999814987183]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it.

In [15]:
probability_model = tf.keras.Sequential([
                                         model,
                                         tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.1705710e-07, 7.6058356e-08, 2.1122547e-05, 9.3223272e-05,
        2.2920511e-12, 8.3462373e-07, 1.9958697e-13, 9.9987853e-01,
        2.6665978e-07, 5.8387250e-06],
       [2.9329152e-08, 3.6228088e-05, 9.9993587e-01, 1.1091690e-05,
        8.8246103e-15, 3.7095984e-07, 4.8631949e-07, 3.0004777e-14,
        1.5976224e-05, 8.7806854e-12],
       [2.2550410e-06, 9.9667692e-01, 7.5296564e-05, 3.6092079e-05,
        1.1313955e-04, 1.0904665e-06, 4.2767861e-06, 2.8565661e-03,
        2.3346453e-04, 9.7028158e-07],
       [9.9998271e-01, 1.1969929e-09, 7.0720321e-06, 1.1972767e-08,
        4.7272085e-08, 1.5581026e-06, 2.6133625e-06, 5.7059374e-06,
        7.1974817e-09, 3.9191823e-07],
       [1.2258460e-05, 9.7180459e-09, 1.6595171e-05, 1.2539255e-07,
        9.9585670e-01, 2.1528386e-07, 3.5466446e-06, 2.9665885e-05,
        1.2204991e-06, 4.0796548e-03]], dtype=float32)>

# Quickstart for experts

Import Tensorflow to your program.

In [17]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Load and prepare the MNIST dataset.

In [18]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

Use `tf.data` to batch and shuffle the dataset.

In [20]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the tf.keras model using the Keras model subclassing API.

In [21]:
class MyModel(Model):
  
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

In [22]:
# Create an instance of the model
model = MyModel()

Choose an optimiser and loss function for training.

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [25]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model.

In [26]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model.

In [31]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [32]:
EPOCHS = 5

In [33]:
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 0.03707418590784073, Accuracy: 98.84166717529297, Test Loss: 0.055687546730041504, Test Accuracy: 98.18000030517578
Epoch 2, Loss: 0.018723540008068085, Accuracy: 99.37333679199219, Test Loss: 0.05161529779434204, Test Accuracy: 98.44999694824219
Epoch 3, Loss: 0.012314803898334503, Accuracy: 99.58000183105469, Test Loss: 0.05714106187224388, Test Accuracy: 98.4000015258789
Epoch 4, Loss: 0.007780862972140312, Accuracy: 99.73333740234375, Test Loss: 0.06765719503164291, Test Accuracy: 98.27999877929688
Epoch 5, Loss: 0.00678337411954999, Accuracy: 99.7733383178711, Test Loss: 0.07473964244127274, Test Accuracy: 98.25
